# This file is based on the data of hw3
## We mainly do two things:
-  Classify data
- Train the regression model



In [1]:
# check python version
# must be 3.8 to 3.10
from platform import python_version

print(python_version())

3.8.2


In [2]:
# install packages
!pip3 install polygon-api-client --user
!pip3 install sqlalchemy --user
!pip3 install pandas --user
!pip3 install numpy==1.22.3 --user
!pip3 install scikit-learn==0.23.2 --user
!pip3 install pycaret --user

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Read Data Logic
- Read data from csv files in day1 to dataframe
- Sort df by vol and fd
- Label the vol and fd columns by 1, 2, 3

In [185]:
import pandas as pd

In [186]:
# read data from csv
columns = ["time","mean", "std", "max", "min", "vol", "fd","r"]
AUDUSD_df = pd.read_csv('./data-ten-hours/AUDUSD_agg.csv', names=columns, na_values='?', header=None,dtype={"time":'str',"mean":'float64', "std":'float64', "max":'float64', "min":'float64', "vol":'float64', "fd":'float64',"r":'float64'})
EURUSD_df = pd.read_csv('./data-ten-hours/EURUSD_agg.csv', names=columns, na_values='?', header=None)
GBPEUR_df = pd.read_csv('./data-ten-hours/GBPEUR_agg.csv', names=columns, na_values='?', header=None)
USDCAD_df = pd.read_csv('./data-ten-hours/USDCAD_agg.csv', names=columns, na_values='?', header=None)
USDCNY_df = pd.read_csv('./data-ten-hours/USDCNY_agg.csv', names=columns, na_values='?', header=None)
USDCZK_df = pd.read_csv('./data-ten-hours/USDCZK_agg.csv', names=columns, na_values='?', header=None)
USDINR_df = pd.read_csv('./data-ten-hours/USDINR_agg.csv', names=columns, na_values='?', header=None)
USDJPY_df = pd.read_csv('./data-ten-hours/USDJPY_agg.csv', names=columns, na_values='?', header=None)
USDMXN_df = pd.read_csv('./data-ten-hours/USDMXN_agg.csv', names=columns, na_values='?', header=None)
USDPLN_df = pd.read_csv('./data-ten-hours/USDPLN_agg.csv', names=columns, na_values='?', header=None)

In [187]:
# match the currency pairs and data frame
currency_pairs_dict = {"AUDUSD": AUDUSD_df, "EURUSD": EURUSD_df, "GBPEUR": GBPEUR_df, "USDCAD": USDCAD_df,
                       "USDCNY": USDCNY_df, "USDCZK": USDCZK_df, "USDINR": USDINR_df, "USDJPY": USDJPY_df,
                       "USDMXN": USDMXN_df, "USDPLN": USDPLN_df}

In [188]:
# classify the data by vol/FD
# set them as 1=low, 2=medium, 3=high
# first bar=top 33
# second bar=top=67
def classify(currency, original_df, column_name,first_bar=33, second_bar=67):
    df=original_df.copy(deep=True)
    # sort by vol
    df=df.sort_values(by=[column_name])
    # first_bar_dict[column_name]
    for i, row in df.iterrows():
        if i < first_bar:
            df.at[i,column_name] = 1
        elif i < second_bar:
            df.at[i,column_name] = 2
        else:
            df.at[i,column_name] = 3
    return df

In [189]:
formatted_currency_pairs_dict={}
# classify all the fds
for key, df in currency_pairs_dict.items():
    # classify by fd
    df=classify(key,df,"fd")
    # classify by vol
    df=classify(key,df,"vol")
    # show the results to check
    print(df)
    formatted_currency_pairs_dict[key]=df

                   time      mean       std       max       min  vol   fd  \
8   2022-12-05 16:47:22  0.669709  0.000045  0.669900  0.669600  1.0  1.0   
3   2022-12-05 16:01:05  0.669215  0.000064  0.669400  0.669000  1.0  1.0   
5   2022-12-05 16:19:36  0.669362  0.000073  0.669600  0.669200  1.0  1.0   
4   2022-12-05 16:10:17  0.669341  0.000082  0.669500  0.669100  1.0  1.0   
9   2022-12-05 16:56:13  0.669793  0.000052  0.670000  0.669600  1.0  1.0   
..                  ...       ...       ...       ...       ...  ...  ...   
49  2022-12-05 22:55:28  0.672830  0.000288  0.673300  0.671800  2.0  2.0   
16  2022-12-05 17:58:54  0.670178  0.000126  0.671700  0.669940  1.0  1.0   
14  2022-12-05 17:41:03  0.670112  0.000325  0.671955  0.669900  1.0  1.0   
47  2022-12-05 22:38:15  0.672557  0.000541  0.673535  0.671345  2.0  2.0   
11  2022-12-05 17:14:14  0.670027  0.000298  0.672625  0.669700  1.0  1.0   

           r  
8   0.000223  
3   0.000137  
5   0.000031  
4   0.000188  


# Train Models

In [190]:
from pycaret import regression

In [191]:
# extract features we want to use to train the model
def extract_features(df):
    df=df[["mean","fd","vol","r"]]
    return df

In [192]:
# train model for all currency df
def train_regression_model(df, target):
    regression.setup(data=df,target=target,fold_shuffle=True)
    best=regression.compare_models()
    regression.evaluate_model(best)
    return best

In [193]:
currency_model_dict={}
for curreny, df in formatted_currency_pairs_dict.items():
    # extract feature
    df=extract_features(df)
    currency_model_dict[curreny]=train_regression_model(df,'r')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,0.0003,0.0000,0.0004,-0.2237,0.0003,3.5783,0.0070
en,Elastic Net,0.0003,0.0000,0.0004,-0.2237,0.0003,3.5783,0.0070
dummy,Dummy Regressor,0.0003,0.0000,0.0004,-0.2237,0.0003,3.5783,0.0060
llar,Lasso Least Angle Regression,0.0003,0.0000,0.0004,-0.2237,0.0003,3.5783,0.0070
ridge,Ridge Regression,0.0003,0.0000,0.0004,-0.2533,0.0004,4.2343,0.0070
br,Bayesian Ridge,0.0003,0.0000,0.0004,-0.2542,0.0003,4.2182,0.0070
omp,Orthogonal Matching Pursuit,0.0003,0.0000,0.0004,-0.2877,0.0003,1.8491,0.0070
par,Passive Aggressive Regressor,0.0003,0.0000,0.0004,-0.3206,0.0004,1.0000,0.0080
lar,Least Angle Regression,0.0003,0.0000,0.0004,-0.3287,0.0003,2.9927,0.0070
lr,Linear Regression,0.0003,0.0000,0.0004,-0.3287,0.0003,2.9927,0.0070


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

# Perdict one hour data with models trained before

In [194]:
# Read one hour data to test
columns = ["time","mean", "std", "max", "min", "vol", "fd","r"]
AUDUSD_test_df = pd.read_csv('./data-one-hour/AUDUSD_agg.csv', names=columns, na_values='?', header=None,dtype={"time":'str',"mean":'float64', "std":'float64', "max":'float64', "min":'float64', "vol":'float64', "fd":'float64',"r":'float64'})
EURUSD_test_df = pd.read_csv('./data-one-hour/EURUSD_agg.csv', names=columns, na_values='?', header=None)
GBPEUR_test_df = pd.read_csv('./data-one-hour/GBPEUR_agg.csv', names=columns, na_values='?', header=None)
USDCAD_test_df = pd.read_csv('./data-one-hour/USDCAD_agg.csv', names=columns, na_values='?', header=None)
USDCNY_test_df = pd.read_csv('./data-one-hour/USDCNY_agg.csv', names=columns, na_values='?', header=None)
USDCZK_test_df = pd.read_csv('./data-one-hour/USDCZK_agg.csv', names=columns, na_values='?', header=None)
USDINR_test_df = pd.read_csv('./data-one-hour/USDINR_agg.csv', names=columns, na_values='?', header=None)
USDJPY_test_df = pd.read_csv('./data-one-hour/USDJPY_agg.csv', names=columns, na_values='?', header=None)
USDMXN_test_df = pd.read_csv('./data-one-hour/USDMXN_agg.csv', names=columns, na_values='?', header=None)
USDPLN_test_df = pd.read_csv('./data-one-hour/USDPLN_agg.csv', names=columns, na_values='?', header=None)

In [195]:
# match the currency pairs and data frame
currency_pairs_test_dict = {"AUDUSD": AUDUSD_test_df, "EURUSD": EURUSD_test_df, "GBPEUR": GBPEUR_test_df, "USDCAD": USDCAD_test_df,
                       "USDCNY": USDCNY_test_df, "USDCZK": USDCZK_test_df, "USDINR": USDINR_test_df, "USDJPY": USDJPY_test_df,
                       "USDMXN": USDMXN_test_df, "USDPLN": USDPLN_test_df}

In [198]:
print(currency_pairs_dict["AUDUSD"])
# print(currency_pairs_test_dict["AUDUSD"])

                  time      mean       std       max       min       vol  \
0  2022-12-06 13:12:08  0.669034  0.000120  0.669300  0.668815  0.000485   
1  2022-12-06 13:22:37  0.668671  0.000137  0.669000  0.668470  0.000530   
2  2022-12-06 13:32:26  0.668413  0.000139  0.668700  0.668135  0.000565   
3  2022-12-06 13:42:33  0.668596  0.000230  0.669000  0.668100  0.000900   
4  2022-12-06 13:53:06  0.668813  0.000050  0.669000  0.668700  0.000300   
5  2022-12-06 14:03:13  0.668724  0.000079  0.668900  0.668565  0.000335   
6  2022-12-06 14:13:38  0.668678  0.000070  0.668845  0.668500  0.000345   
7  2022-12-06 14:24:23  0.668940  0.000171  0.669200  0.668600  0.000600   
8  2022-12-06 14:35:27  0.668890  0.000256  0.669300  0.668400  0.000900   
9  2022-12-06 14:45:51  0.668827  0.000205  0.669320  0.668500  0.000820   

             fd         r  
0  0.000000e+00  0.000000  
1  2.003774e+07 -0.000543  
2  1.210973e+07 -0.000386  
3  7.288889e+06  0.000274  
4  1.100333e+07  0.0003

In [199]:
# classify test data 
def classify_test_data(currency,column_name):
    # get original df
    train_df=currency_pairs_dict[currency]
    # sort by vol
    train_df=train_df.sort_values(by=[column_name])
    # get first bar and second bar
    first=train_df.at[33,column_name]
    second=train_df.at[67,column_name]
    for i, row in currency_pairs_test_dict[currency].iterrows():
        if i < first:
            df.at[i,column_name] = 1
        elif i<second:
            df.at[i,column_name]=2
        else:
            df.at[i,column_name] = 3

In [200]:
# classify test df
for key,df in currency_pairs_test_dict.items():
    # classify by fd
    classify_test_data(key,"fd")
    # classify by vol
    classify_test_data(key,"vol")
    # show the results to check
    currency_pairs_test_dict[key]=df

In [201]:
print(currency_pairs_test_dict["AUDUSD"])

                  time      mean       std       max       min  vol   fd  \
0  2022-12-06 13:12:08  0.669034  0.000120  0.669300  0.668815  1.0  1.0   
1  2022-12-06 13:22:37  0.668671  0.000137  0.669000  0.668470  3.0  1.0   
2  2022-12-06 13:32:26  0.668413  0.000139  0.668700  0.668135  3.0  1.0   
3  2022-12-06 13:42:33  0.668596  0.000230  0.669000  0.668100  3.0  1.0   
4  2022-12-06 13:53:06  0.668813  0.000050  0.669000  0.668700  3.0  1.0   
5  2022-12-06 14:03:13  0.668724  0.000079  0.668900  0.668565  3.0  1.0   
6  2022-12-06 14:13:38  0.668678  0.000070  0.668845  0.668500  3.0  1.0   
7  2022-12-06 14:24:23  0.668940  0.000171  0.669200  0.668600  3.0  1.0   
8  2022-12-06 14:35:27  0.668890  0.000256  0.669300  0.668400  3.0  1.0   
9  2022-12-06 14:45:51  0.668827  0.000205  0.669320  0.668500  3.0  1.0   

          r  
0  0.000000  
1 -0.000543  
2 -0.000386  
3  0.000274  
4  0.000325  
5 -0.000134  
6 -0.000068  
7  0.000391  
8 -0.000074  
9 -0.000095  


In [202]:
# extract feature to test
def extract_test_features(df):
    return df[["mean","fd","vol"]]

In [203]:
currency_perdictions_dict={}
# perdict
for key,value in currency_model_dict.items():
    test_df=extract_test_features(currency_pairs_test_dict[key])
    perdictions=regression.predict_model(value,data=test_df)
    currency_perdictions_dict[key]=perdictions

In [217]:
# extract and join actual and perdicted return
output_dict={}
for key, df in currency_perdictions_dict.items():
    # extract label
    perdict_return=df["Label"]
    acutal_return=currency_pairs_test_dict[key]["r"]
    out=pd.concat([perdict_return, acutal_return], axis=1)
    output_dict[key]=out
    

In [221]:
# write to csv
for key,df in output_dict.items():
    df["error"]=df['Label']-df['r']
    df.to_csv('./final_output/'+key+".csv",header=['perdicted_return','acutal_return','error'])